In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import ydata_profiling
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import GridSearchCV
import xgboost
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files
uploades = files.upload()

Saving abalone_preprocessed.csv to abalone_preprocessed.csv


In [ ]:
df = pd.read_csv('abalone_preprocessed.csv')

In [ ]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age,13.5_F,13.5_I,13.5_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,1,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,0,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,0,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,1,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,0,0,1,0


In [ ]:
X = df.drop(['Age'],axis=1)
y = df['Age']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3 ,random_state=42)

In [ ]:
param_grid = [  {'learning_rate': [0.01,0.001,0.1,0.2,0.02], 'max_depth':range(0,10,1),
                 }]

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=10)
# from lightgbm import LightGBMClassifier
xgb = xgboost.XGBClassifier()
grid_search = GridSearchCV(xgb, param_grid, cv=cv, return_train_score=True, n_jobs = -1)

In [ ]:
import IPython
from google.colab import output
display(IPython.display.Javascript('''
function ClickConnect()
  btn == document. querySelector("colab-connect-button")
  if (btn ! = null) {
    console. log("Click colab-connect-button");
    btn.click()
    }
  btn = document.getElementById(ok')
  if (btn != null) {
    console. .log("Click reconnect");
    btn.click()
                               I
setInterval(ClickConnect,60000)
'''))


<IPython.core.display.Javascript object>

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 3}

In [ ]:
param_grid = [  {'learning_rate': [0.1], 'max_depth':[3],'max_leaves' : range(10,50),
                 }]
# from lightgbm import LightGBMClassifier
xgb = xgboost.XGBClassifier()
grid_search = GridSearchCV(xgb, param_grid, cv=cv, return_train_score=True, n_jobs = -1)

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'max_leaves': 10}

In [ ]:
param_grid = [  {'learning_rate': [0.1], 'max_depth':[3],'max_leaves' : [10],'min_child_weight':range(0,10,1)
                 }]
xgb = xgboost.XGBClassifier()
grid_search = GridSearchCV(xgb, param_grid, cv=cv, return_train_score=True, n_jobs = -1)

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'max_leaves': 10, 'min_child_weight': 7}

In [ ]:
param_grid = [  {'learning_rate': [0.1], 'max_depth':[3],'max_leaves' : [10],'min_child_weight':[7],
                 'subsample':[0.8,0.9,1],'colsample_bytree':[0.8,0.9,1],'gamma':[0,0.01,0.02],
                 }]
xgb = xgboost.XGBClassifier()
grid_search = GridSearchCV(xgb, param_grid, cv=cv, return_train_score=True, n_jobs = -1)

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0.01,
 'learning_rate': 0.1,
 'max_depth': 3,
 'max_leaves': 10,
 'min_child_weight': 7,
 'subsample': 0.9}

In [ ]:
grid_search.best_score_

0.8006865706591734

In [ ]:
best_xgb = grid_search.best_estimator_
best_xgb.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.01, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=10,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
pred_y = best_xgb.predict(X_test)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, precision_score, classification_report,f1_score, recall_score, confusion_matrix
accuracy = accuracy_score(y_test, pred_y)
print(accuracy)
recall = recall_score(y_test, pred_y)
print(recall)
precision = precision_score(y_test, pred_y)
print(precision)
f1 = f1_score(y_test, pred_y)
print(f1)
print('Classification report for XGB Model:\n',classification_report(y_test, pred_y))

0.7950558213716108
0.8264462809917356
0.7668711656441718
0.7955449482895784
Classification report for XGB Model:
               precision    recall  f1-score   support

           0       0.83      0.77      0.79       649
           1       0.77      0.83      0.80       605

    accuracy                           0.80      1254
   macro avg       0.80      0.80      0.80      1254
weighted avg       0.80      0.80      0.80      1254



In [ ]:
print(confusion_matrix(y_test, pred_y))

[[497 152]
 [105 500]]
